In [1]:
import time
import torch
from torch import nn, optim

import d2lzh_pytorch as d2l

# 将gpu设置为device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

查看gpu是否可用可使用如下代码

In [2]:
torch.cuda.is_available()

True

查看gpu信息（如果有多个gpu改变参数值查看）

In [3]:
torch.cuda.get_device_name(0)

'GeForce GTX 1060 6GB'

以下定义LeNet，LeNet由两个大模块组成，即卷积层和全连接层，其中卷积层又包括两次卷积计算和池化计算（原论文并没有用pooling这个词，而是subsampling?）

**重点：**此处原文中并没有提到，但可以确认的是，下面的各层的参数设置是基于**fashion-mnist**数据集的，尽管之前一直用的都是这个数据集，但我觉得还是应当强调一下，不然参数的设置实在无法解释清楚

为什么全连接层的输入是4x4x16？  
这是输入的28x28x1的图像，经过整个卷积层后得到的大小

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            # nn.Conv2d(1, 6, 5)中，这三个参数分别代表：in_channels, out_channels, kernel_size
            # out_channels即是number of kernels
            nn.Conv2d(1, 6, 5),  # 首先，这里进入通道数为1，表明为灰度图像，符合fashion-mnist情况，如果是RGB图像则需要3通道。此处输出24x24x6
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 2x2的池化层，非常常见 -> 直接结果：高度和宽度缩减一半。此处输出12x12x6
            nn.Conv2d(6, 16, 5),  # 此处输出8x8x16（由于宽和高缩减，增加通道数以保持与上个卷积层参数尺寸相似）
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # 此处输出4x4x16
        )
        
        self.fc = nn.Sequential(
            nn.Linear(4*4*16, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # softmax函数，获得最终预测概率
        )
    
    def forward(self, img):
        feature = self.conv(img)
        # 卷积层块的输出形状为(批量大小, 通道, 高, 宽)，这里的view()起到让向量扁平的作用
        output = self.fc(feature.view(img.shape[0], -1))  
        return output

将上述定义的网络实例化，查看每个层的形状

**注：**可以发现这样定义网络的话层内各部分都是以数字命名，缺乏解释性，可将上述nn.Sequential内的内容用OrderedDict改写

In [5]:
net = LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


导入fashion-mnist数据集

In [6]:
batch_size = 256

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

重写evaluate_accuracy以及训练函数，以支持GPU运算

In [7]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, nn.Module):  # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        if isinstance(net, nn.Module):
            net.eval()  # 评估模式下，Dropout被关闭，BatchNorm使用训练中得到的均值与方差进行预测
            # 这里的加法中加入float()并没有问题，只是acc_sum本身就为float型，加法过程中会自动转换的
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).sum().item()
        else:  # 自定义的模型
            if 'is_training' in net.__code__.co_varnames:  # 如果有is_training这个参数，设置为False
                acc_sum += (net(X, is_training=False).argmax(dim=1) == y).sum().item()
            else:
                acc_sum += (net(X).argmax(dim=1) == y).sum().item()
        n += y.shape[0]
    return acc_sum / n

In [8]:
def train_ch5(net, train_iter, test_iter, batch_size, loss, optimizer, device, num_epochs):
    net = net.to(device)  # 转移到GPU上进行运算
    print("training on", device)
    
    for epoch in range(1, num_epochs + 1):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            
            optimizer.zero_grad()
            
            l.backward()
            optimizer.step()
            
            # 原Pytorch代码为
            # train_l_sum += l.cpu().item()
            # train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            # 加上cpu()有必要吗？为什么要把这几个值搬运回CPU上呢？  在我的几次试验中，并无明显提升，上面evaluate_accuracy函数同理
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            
            n += y.shape[0]
            batch_count += 1
        
        test_acc = evaluate_accuracy(test_iter, net)
        
        print("epoch %d, train loss %.4f, train acc %.3f, test acc %.3f, time elapsed %.2f sec"
              % (epoch, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

 学习率采用0.001，训练算法使用Adam算法，损失函数使用交叉熵损失函数。

In [9]:
lr, num_epochs = 0.001, 5
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)  # 用了Adam优化器
train_ch5(net, train_iter, test_iter, batch_size, loss, optimizer, device, num_epochs)

training on cuda
epoch 1, train loss 1.0036, train acc 0.630, test acc 0.714, time elapsed 6.75 sec
epoch 2, train loss 0.6390, train acc 0.756, test acc 0.774, time elapsed 5.93 sec
epoch 3, train loss 0.5553, train acc 0.790, test acc 0.788, time elapsed 5.87 sec
epoch 4, train loss 0.4973, train acc 0.815, test acc 0.799, time elapsed 5.80 sec
epoch 5, train loss 0.4590, train acc 0.833, test acc 0.829, time elapsed 5.82 sec


总结

- 卷积神经网络就是含卷积层的网络
- LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类